In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import urllib

server = 'auqui.database.windows.net'
port = '1433'
database = 'dbnormativo'
username = 'administrador'
password = 'Auqui$2020'
driver = '{ODBC Driver 17 for SQL Server}'                     

#connection = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)
conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

params = urllib.parse.quote_plus(conn)
conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import sqlalchemy
from sqlalchemy import create_engine

class PrcRcdClasificacionAlineada():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaClasificacionAlineada(self):
    
    fechaProcesoRCC = self.fecha_proceso.replace(day=1) - timedelta(days=1)
    fechaProcesoRCC_str = fechaProcesoRCC.strftime('%Y-%m-%d')
    #prueba --borrar
    fechaProcesoRCC_str = '2021-01-31'

    #reproceso
    limpiaClasifAlineada = """
    UPDATE CLIENTE
    SET CLASIFICACION_DEUDOR_CON_ALINEAMIENTO = NULL,
        CLASIFICACION_DEUDOR = CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO
    WHERE FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CLASIFICACION_DEUDOR_CON_ALINEAMIENTO IS NOT NULL;
    """
    self.cursor.execute(limpiaClasifAlineada)
    self.connection.commit()

    eliminaIndicador_pa = ("DELETE CLIENTE_DEUDOR_DETALLE WHERE INDICADOR = 'PA' AND FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
    eliminaIndicador_ce = ("DELETE CLIENTE_DEUDOR_DETALLE WHERE INDICADOR = 'CE' AND FECHA_PROCESO = '" + self.fecha_proceso_str + "'")
    self.cursor.execute(eliminaIndicador_pa)
    self.cursor.execute(eliminaIndicador_ce)
    self.connection.commit()

    #proceso --**revisar el filtro de concepto saldo, se debe agregar los conceptos capital aplicable para falabella
    cliente = """SELECT DISTINCT A.ID CLIENTE_DEUDOR, B.ID CLIENTE, E.CODIGO_EMPRESA_SBS, D.TIPO_DEUDOR, A.CODIGO_DEUDOR_SBS, A.TIPO_DOCUMENTO_TRIBUTARIO, A.DOCUMENTO_TRIBUTARIO, A.TIPO_DOCUMENTO_IDENTIDAD, A.DOCUMENTO_IDENTIDAD, A.TIPO_PERSONA
    FROM CLIENTE_DEUDOR A 
    JOIN CLIENTE B ON A.ID = B.CLIENTE_DEUDOR
    JOIN CLIENTE_PRODUCTO C ON B.ID = C.CLIENTE
    JOIN CLIENTE_PRODUCTO_CONCEPTO F ON C.ID = F.CLIENTE_PRODUCTO
    JOIN TIPO_CREDITO_SBS D ON D.ID = C.TIPO_CREDITO_SBS
    JOIN EMPRESA E ON E.ID = A.EMPRESA
    WHERE B.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND A.EMPRESA = """"'"+ str(self.empresa) +"""' 
    --AND C.SITUACION_CREDITO = 'N'
    AND F.CONCEPTO_SALDO = 38
    """
    sql_cliente = pd.read_sql_query(cliente, self.connection)
    df_cliente = pd.DataFrame(sql_cliente)

    rcc = """SELECT B.ID RCC, D.ID RCC_SALDOS, B.CODIGO_DEUDOR_SBS, B.TIPO_DOCUMENTO_IDENTIDAD, B.DOCUMENTO_IDENTIDAD, B.TIPO_PERSONA, B.TIPO_DOCUMENTO_TRIBUTARIO, B.DOCUMENTO_TRIBUTARIO, D.EMPRESA, D.CUENTA_CONTABLE, D.DIAS_ATRASO, D.IMPORTE, D.CLASIFICACION_DEUDOR
    FROM RCC B
    JOIN RCC_SALDOS D 
    ON B.FECHA_PROCESO = """"'"+ fechaProcesoRCC_str +"""'
    AND  B.ID = D.RCC
    """
    sql_rcc = pd.read_sql_query(rcc, self.connection)
    df_rcc = pd.DataFrame(sql_rcc)

    ClienteRccSaldos = pd.merge(df_cliente,df_rcc, on = ['CODIGO_DEUDOR_SBS'], how='inner')
    ClienteRccSaldos1 = pd.DataFrame(ClienteRccSaldos)
    ClienteRccSaldos1.drop(['TIPO_DOCUMENTO_IDENTIDAD_y','DOCUMENTO_IDENTIDAD_y', 'TIPO_PERSONA_y', 'TIPO_DOCUMENTO_TRIBUTARIO_y', 'DOCUMENTO_TRIBUTARIO_y'], axis="columns", inplace=True) 
    
    ClienteRccSaldos1.rename(columns={'TIPO_DOCUMENTO_TRIBUTARIO_x':'TIPO_DOCUMENTO_TRIBUTARIO',
                                    'DOCUMENTO_TRIBUTARIO_x':'DOCUMENTO_TRIBUTARIO',
                                    'TIPO_DOCUMENTO_IDENTIDAD_x':'TIPO_DOCUMENTO_IDENTIDAD',
                                    'DOCUMENTO_IDENTIDAD_x':'DOCUMENTO_IDENTIDAD',
                                    'TIPO_PERSONA_x':'TIPO_PERSONA'}, inplace=True)
    rfClienteRccSaldos1 = pd.DataFrame(ClienteRccSaldos1)

    ClienteRccSaldos2 = pd.merge(df_cliente,df_rcc, on = ['TIPO_DOCUMENTO_IDENTIDAD', 'DOCUMENTO_IDENTIDAD','TIPO_PERSONA'], how='inner')
    ClienteRccSaldos2.drop(['CODIGO_DEUDOR_SBS_y','TIPO_DOCUMENTO_TRIBUTARIO_y', 'DOCUMENTO_TRIBUTARIO_y'], axis="columns", inplace=True) 
    ClienteRccSaldos2.rename(columns={'CODIGO_DEUDOR_SBS_x':'CODIGO_DEUDOR_SBS',
                                    'TIPO_DOCUMENTO_TRIBUTARIO_x':'TIPO_DOCUMENTO_TRIBUTARIO',
                                    'DOCUMENTO_TRIBUTARIO_x':'DOCUMENTO_TRIBUTARIO'}, inplace=True)

    ClienteRccSaldos3 = pd.merge(df_cliente,df_rcc, on = ['TIPO_DOCUMENTO_TRIBUTARIO', 'DOCUMENTO_TRIBUTARIO'], how='inner')
    ClienteRccSaldos3.drop(['CODIGO_DEUDOR_SBS_y','TIPO_DOCUMENTO_IDENTIDAD_y','DOCUMENTO_IDENTIDAD_y', 'TIPO_PERSONA_y'], axis="columns", inplace=True) 
    ClienteRccSaldos3.rename(columns={'CODIGO_DEUDOR_SBS_x':'CODIGO_DEUDOR_SBS',
                                    'TIPO_DOCUMENTO_IDENTIDAD_x':'TIPO_DOCUMENTO_IDENTIDAD',
                                    'DOCUMENTO_IDENTIDAD_x':'DOCUMENTO_IDENTIDAD',
                                    'TIPO_PERSONA_x':'TIPO_PERSONA'}, inplace=True)

    #unimos las consultas y eliminamos los duplicados
    dfClienteRccSaldos = pd.concat([ClienteRccSaldos1,ClienteRccSaldos2,ClienteRccSaldos3]).drop_duplicates()

    #Obtenemos la cantidad de entidad financieras asociadas a un cliente
    ClienteCantidadEmpresas = dfClienteRccSaldos[['CLIENTE_DEUDOR','RCC','EMPRESA']].drop_duplicates()
    
    #--**mejorar codigo**--
    dfClienteRccSaldos.to_sql('CLIENTE_RCC_SALDOS', con=conn_str, if_exists='replace',index=False)
    ClienteCantidadEmpresas.to_sql('CLIENTE_CANT_EMPRESAS', con=conn_str, if_exists='replace',index=False)
    
    ClienteRccSaldosResumen = """SELECT A.CLIENTE, A.CLIENTE_DEUDOR, A.RCC, A.CODIGO_EMPRESA_SBS, A.EMPRESA, A.TIPO_DEUDOR, A.CLASIFICACION_DEUDOR, 
    MAX(DIAS_ATRASO) DIAS_ATRASO_MAYOR, SUM(cast( IMPORTE as float)) IMPORTE
    FROM CLIENTE_RCC_SALDOS A 
    JOIN RCC_CUENTA_CONTABLE B
    ON B.CUENTA_CONTABLE = SUBSTRING(A.CUENTA_CONTABLE,1,LEN(B.CUENTA_CONTABLE))
    AND EXISTS(SELECT 1 FROM CLIENTE_CANT_EMPRESAS C WHERE A.RCC = C.RCC GROUP BY C.RCC HAVING COUNT(1) > 1)
    GROUP BY A.CLIENTE, A.CLIENTE_DEUDOR, A.RCC, A.CODIGO_EMPRESA_SBS, A.EMPRESA, A.TIPO_DEUDOR, A.CLASIFICACION_DEUDOR
    """
    sql_ClienteRccSaldosResumen = pd.read_sql_query(ClienteRccSaldosResumen, self.connection)
    df_ClienteRccSaldosResumen = pd.DataFrame(sql_ClienteRccSaldosResumen)
    df_ClienteRccSaldosResumen.to_sql('CLIENTE_RCC_SALDOS_RESUMEN', con=conn_str, if_exists='replace',index=False)

    ActualizaClasifAlineada = """;WITH X AS
    (SELECT A.*,
            SUM(CASE WHEN A.TIPO_DEUDOR = 2 AND A.DIAS_ATRASO_MAYOR <= 1800 THEN A.IMPORTE
                    WHEN A.TIPO_DEUDOR = 1 THEN A.IMPORTE ELSE 0 END) OVER(PARTITION BY A.RCC) IMPORTE_DEUDOR,
            SUM(CASE WHEN A.TIPO_DEUDOR = 2 AND A.DIAS_ATRASO_MAYOR <= 1800 THEN A.IMPORTE
                    WHEN A.TIPO_DEUDOR = 1 THEN A.IMPORTE ELSE 0 END) OVER(PARTITION BY A.RCC, A.EMPRESA) IMPORTE_DEUDOR_EMPRESA
    FROM CLIENTE_RCC_SALDOS_RESUMEN A),
    Y AS 
    (SELECT X.*,
            RANK() OVER(PARTITION BY X.RCC ORDER BY X.CLASIFICACION_DEUDOR DESC) RKING 
    FROM X
    WHERE CAST(X.EMPRESA AS INT) != CAST(X.CODIGO_EMPRESA_SBS AS INT)
    AND CASE WHEN IMPORTE_DEUDOR_EMPRESA > 0 AND IMPORTE_DEUDOR > 0 THEN IMPORTE_DEUDOR_EMPRESA/IMPORTE_DEUDOR ELSE 0 END >= .2)
    UPDATE A
    SET A.CLASIFICACION_DEUDOR_CON_ALINEAMIENTO = Y.CLASIFICACION_DEUDOR,
    A.CLASIFICACION_DEUDOR = C.CLASIFICACION_DEUDOR
    FROM CLIENTE A JOIN Y ON A.ID = Y.CLIENTE
    JOIN DISCREPANCIA_CLASIFICACION_DEUDOR C ON C.TIPO_DEUDOR = Y.TIPO_DEUDOR AND C.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO = A.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO AND C.CLASIFICACION_DEUDOR_CON_ALINEAMIENTO = Y.CLASIFICACION_DEUDOR
    WHERE Y.RKING = 1 
    AND CASE WHEN Y.TIPO_DEUDOR = 2 AND Y.CLASIFICACION_DEUDOR IN (3,4) THEN 1 WHEN Y.TIPO_DEUDOR = 1 THEN 1 ELSE 0 END = 1;
    """
    self.cursor.execute(ActualizaClasifAlineada)

    insertaIndicador = """;WITH X AS
    (SELECT A.*,
            SUM(CASE WHEN A.TIPO_DEUDOR = 2 AND A.DIAS_ATRASO_MAYOR <= 1800 THEN A.IMPORTE
                    WHEN A.TIPO_DEUDOR = 1 THEN A.IMPORTE ELSE 0 END) OVER(PARTITION BY A.RCC) IMPORTE_DEUDOR,
            (CASE WHEN A.TIPO_DEUDOR = 2 AND A.DIAS_ATRASO_MAYOR <= 1800 THEN A.IMPORTE
                    WHEN A.TIPO_DEUDOR = 1 THEN A.IMPORTE ELSE 0 END) IMPORTE_CLASIFICACION
    FROM CLIENTE_RCC_SALDOS_RESUMEN A),
    Y AS
    (SELECT CLIENTE_DEUDOR, CLASIFICACION_DEUDOR, SUM(IMPORTE_CLASIFICACION) IMPORTE_CLASIFICACION, IMPORTE_DEUDOR
    FROM X 
    GROUP BY CLIENTE_DEUDOR, CLASIFICACION_DEUDOR, IMPORTE_DEUDOR)
    INSERT INTO CLIENTE_DEUDOR_DETALLE(CLIENTE_DEUDOR, INDICADOR, FECHA_INDICADOR, IMPORTE, VALOR, FECHA_PROCESO)
    SELECT CLIENTE_DEUDOR, 'PA', """"'"+ self.fecha_proceso_str +"""', CASE WHEN IMPORTE_CLASIFICACION > 0 AND IMPORTE_DEUDOR>0 THEN ROUND(IMPORTE_CLASIFICACION/IMPORTE_DEUDOR,2) ELSE 0 END, CLASIFICACION_DEUDOR, """"'"+ self.fecha_proceso_str +"""' 
    FROM Y
    INSERT INTO CLIENTE_DEUDOR_DETALLE(CLIENTE_DEUDOR, INDICADOR, FECHA_INDICADOR, IMPORTE, VALOR, FECHA_PROCESO)
    SELECT A.CLIENTE_DEUDOR, 'CE', """"'"+ self.fecha_proceso_str +"""', COUNT(1), NULL, """"'"+ self.fecha_proceso_str +"""' FROM #ClienteCantidadEmpresas A GROUP BY A.CLIENTE_DEUDOR
    """
    self.cursor.execute(insertaIndicador)
    self.connection.commit()
    #--**mejorar codigo**--


#Ejecucion prueba
objeto = PrcRcdClasificacionAlineada(1)
objeto.ProcesaClasificacionAlineada()